Neural Art Style Transfer
=============

(This demo is adapted from [neuralart_tensorflow](https://github.com/ckmarkoh/neuralart_tensorflow) and [this paper review](https://github.com/tensorflow/magenta/blob/master/magenta/reviews/styletransfer.md))

Introduction
----------
In late August 2015, Gatys et al. from The University of Tübingen published [A Neural Algorithm of Artistic Style](http://arxiv.org/pdf/1508.06576v2.pdf). It demonstrated a way to present one piece of artwork in the style of a separate piece.  The stunning results gained a lot of public attention. https://deepdreamgenerator.com/ Provides a platform for creating Deep Dream & Deep Style arts without writing code.

The style transfer model is built on top of a deep conv net: [**VGG19**](http://www.robots.ox.ac.uk/~vgg/research/very_deep/), which is used in the ImageNet Large-Scale Visual Recognition Challenge (ILSVRC). VGG19 is trained on more than a million images and can classify images into 1000 object categories. One key insight is that such networks trained to perform object detection end up learning to separate content from style.

At a high level, the general technique is pretty straightforward. First, define a function that describes the quality of a generated image intended to match a particular content and style. Then, synthesize an image that maximizes that score. This is done by starting with a noisy content image and then optimizing using backpropagation, just like what’s done for training neural nets.



Requirements
---------
(Only if you want to follow along)

This demo requires a copy of the [pretrained VGG19 model data](https://drive.google.com/file/d/0B8QJdgMvQDrVU2cyZjFKU1RrLUU/view?usp=sharing) placed in the same directory.

GPU is recommended for this task. With a GPU it should take several miniutes to get meaningful results, wheares with CPU only it may take several hours.

Tensorflow Implementation
---------

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
import scipy.misc
import os

Image related utils. A image is represented as a tensor of shape `(1, IMAGE_H, IMAGE_W, 3)`.

In [ ]:
IMAGE_W = 400  # Small fixed image size for demo
IMAGE_H = 300 
OUTOUT_DIR = './results'

MEAN_VALUES = np.array([123, 117, 104]).reshape((1,1,1,3))  # RGB means

def read_image(path):
    image = scipy.misc.imread(path)
    image = image[np.newaxis,:IMAGE_H,:IMAGE_W,:] 
    image = image - MEAN_VALUES
    return image

def write_image(path, image):
    image = image + MEAN_VALUES
    image = image[0]
    image = np.clip(image, 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)



Functions for rebuilding the vgg network from a file with pretrained parameters. The parameters of the conv net layers will not change during training.

In [ ]:
def build_net(ntype, nin, nwb=None):
    if ntype == 'conv':
        return tf.nn.relu(tf.nn.conv2d(nin, nwb[0], strides=[1, 1, 1, 1],
                                       padding='SAME') + nwb[1])
    elif ntype == 'pool':
        return tf.nn.avg_pool(nin, ksize=[1, 2, 2, 1],
                                    strides=[1, 2, 2, 1], padding='SAME')

def get_weight_bias(vgg_layers, i,):
    weights = vgg_layers[i][0][0][0][0][0]
    weights = tf.constant(weights)
    bias = vgg_layers[i][0][0][0][0][1]
    bias = tf.constant(np.reshape(bias, (bias.size)))
    return weights, bias

def build_vgg19(path):
    net = {}
    vgg_rawnet = scipy.io.loadmat(path)
    vgg_layers = vgg_rawnet['layers'][0]
    net['input'] = tf.Variable(np.zeros((1, IMAGE_H, IMAGE_W, 3)).astype('float32'))
    net['conv1_1'] = build_net('conv',net['input'],get_weight_bias(vgg_layers,0))
    net['conv1_2'] = build_net('conv',net['conv1_1'],get_weight_bias(vgg_layers,2))
    net['pool1']   = build_net('pool',net['conv1_2'])
    net['conv2_1'] = build_net('conv',net['pool1'],get_weight_bias(vgg_layers,5))
    net['conv2_2'] = build_net('conv',net['conv2_1'],get_weight_bias(vgg_layers,7))
    net['pool2']   = build_net('pool',net['conv2_2'])
    net['conv3_1'] = build_net('conv',net['pool2'],get_weight_bias(vgg_layers,10))
    net['conv3_2'] = build_net('conv',net['conv3_1'],get_weight_bias(vgg_layers,12))
    net['conv3_3'] = build_net('conv',net['conv3_2'],get_weight_bias(vgg_layers,14))
    net['conv3_4'] = build_net('conv',net['conv3_3'],get_weight_bias(vgg_layers,16))
    net['pool3']   = build_net('pool',net['conv3_4'])
    net['conv4_1'] = build_net('conv',net['pool3'],get_weight_bias(vgg_layers,19))
    net['conv4_2'] = build_net('conv',net['conv4_1'],get_weight_bias(vgg_layers,21))
    net['conv4_3'] = build_net('conv',net['conv4_2'],get_weight_bias(vgg_layers,23))
    net['conv4_4'] = build_net('conv',net['conv4_3'],get_weight_bias(vgg_layers,25))
    net['pool4']   = build_net('pool',net['conv4_4'])
    net['conv5_1'] = build_net('conv',net['pool4'],get_weight_bias(vgg_layers,28))
    net['conv5_2'] = build_net('conv',net['conv5_1'],get_weight_bias(vgg_layers,30))
    net['conv5_3'] = build_net('conv',net['conv5_2'],get_weight_bias(vgg_layers,32))
    net['conv5_4'] = build_net('conv',net['conv5_3'],get_weight_bias(vgg_layers,34))
    net['pool5']   = build_net('pool',net['conv5_4'])
    return net


Here we define the content loss and style loss.
- Content loss is simply the *L2 difference at a particular layer*. More specifically, for image X and content image C, compute the L2 loss between the output of X and that of C, at the layer conv4_2 in the conv net.
- For style loss, the paper uses [Gram matrices](https://en.wikipedia.org/wiki/Gramian_matrix). Empirically, these are a good proxy for feature correlations, and so the *L2 difference between the Gram matrix* of one image and that of another works well as a way to compare how close they are in style.

In [ ]:
def build_content_loss(p, x):
    M = p.shape[1] * p.shape[2]
    N = p.shape[3]
    loss = (1. / (2 * N**0.5 * M**0.5)) * tf.reduce_sum(tf.pow((x - p), 2))
    return loss

def gram_matrix(x, area, depth):
    x1 = tf.reshape(x,(area,depth))
    g = tf.matmul(tf.transpose(x1), x1)
    return g

def gram_matrix_val(x, area, depth):
    x1 = x.reshape(area,depth)
    g = np.dot(x1.T, x1)
    return g

def build_style_loss(p, x):
    M = p.shape[1] * p.shape[2]
    N = p.shape[3]
    A = gram_matrix_val(p, M, N )
    G = gram_matrix(x, M, N )
    loss = (1. / (4 * N**2 * M**2)) * tf.reduce_sum(tf.pow((G - A), 2))
    return loss


In [ ]:
VGG_MODEL = 'imagenet-vgg-verydeep-19.mat'  # pretrained deep conv net model
INI_NOISE_RATIO = 0.7
STYLE_STRENGTH = 300  # How much to favor style over the original content
ITERATION = 2000

# Layers in the VGG19 conv net for calculating the losses.
CONTENT_LAYERS =[('conv4_2',1.)]
STYLE_LAYERS=[('conv1_1',1.),('conv2_1',1.),('conv3_1',1.),('conv4_1',1.),('conv5_1',1.)]

def stylize(content, style):
    noise_img = np.random.uniform(-20, 20, (1, IMAGE_H, IMAGE_W, 3)).astype('float32')
    content_img = read_image(content)
    style_img = read_image(style)
    
    net = build_vgg19(VGG_MODEL)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        sess.run([net['input'].assign(content_img)])
        
        # build the content loss
        cost_content = sum(map(lambda l,: l[1] * build_content_loss(sess.run(net[l[0]]) ,  net[l[0]])
            , CONTENT_LAYERS))
        
        # build the style loss
        sess.run([net['input'].assign(style_img)])
        cost_style = sum(map(lambda l: l[1] * build_style_loss(sess.run(net[l[0]]) ,  net[l[0]])
            , STYLE_LAYERS))

        # Minimizing a combination of content loss and style loss
        cost_total = cost_content + STYLE_STRENGTH * cost_style
        optimizer = tf.train.AdamOptimizer(2.0)
        train = optimizer.minimize(cost_total)
        sess.run(tf.global_variables_initializer())
        
        # The initial image is a noisy version of the content
        sess.run(net['input'].assign( INI_NOISE_RATIO * noise_img + (1.-INI_NOISE_RATIO) * content_img ))

        if not os.path.exists(OUTOUT_DIR):
                os.mkdir(OUTOUT_DIR)
        print("Training started...")
        
        for i in range(ITERATION + 1):
            sess.run(train)
            if i % 100 == 0:
                result_img = sess.run(net['input'])
                print(str(i) + "\t" + str(sess.run(cost_total)))
                write_image(os.path.join(OUTOUT_DIR,'%s.png'%(str(i).zfill(4))),result_img)
    

Now we are ready to try the algoritm on input images:

In [ ]:
stylize(content="images/Quad.jpg", style="images/StarryNight.jpg")

Style ![Original](images/StarryNight.jpg)
Original ![Original](images/Quad.jpg)
0 ![0](results/0000.png)
100 ![100](results/0100.png)
500 ![100](results/0500.png)
1000 ![1000](results/1000.png)
1500 ![1000](results/1500.png)
2000 ![1000](results/2000.png)